In most of this school, we will be coding with the **Pytorch library**.

It is rather simple to use, and is centred around three main concepts:

* **Tensor** as a central data structure.
* **Auto-differenciation** for implementing the **computation of gradients** in a transparent way for the programmer. 
* Library of state-of-the-art **building blocks** to assemble neural networks. 

(All the following notebook is inspired from the Pytorch documentation)

# The tensor type

The **tensor** data structure is the central one in most DL libraries, and in particular in Pytorch.

Corresponds to the mathematical concept of **tensor** (generalization of matrices, in an arbitrary number of dimensions). 

In [ ]:
import torch
import numpy as np

The same concept can be implemented:

* As a **Python multiple array**.
* As a **Numpy array**.
* As a Pytorch **tensor**.

In [ ]:
python_array  = [[[1, 2], [3, 4]],[[5, 6], [7, 8]]]
numpy_array   = np.array(python_array)
torch_tensor  = torch.from_numpy(numpy_array)
print(python_array)
print(numpy_array)
print(torch_tensor)

There are **many ways to build/initialize a tensor**:

* from scratch,
* from a numpy array,
* as a zero/ones tensor of specified shape,
* as a zero/ones tensor with the characteristics (size) of another tensor,
* from a random number generator,
* ...

In [ ]:
shape              = (2, 3, 2)
torch_tensor_zeros = torch.zeros(shape)
print("torch_tensor_zeros:\n {} \n".format(torch_tensor_zeros))
torch_tensor_ones  = torch.ones(shape)
print("torch_tensor_ones:\n {} \n".format(torch_tensor_ones))
torch_tensor_rand  = torch.rand(shape)
print("torch_tensor_rand:\n {} \n".format(torch_tensor_rand))


In [ ]:
torch_tensor_ones = torch.ones_like(torch_tensor)
print("torch_tensor_ones:\n {} \n".format(torch_tensor_ones))

torch_tensor_zeros = torch.rand_like(torch_tensor_ones, dtype=torch.float) 
print("torch_tensor_rand:\n {} \n".format(torch_tensor_rand))


You can access to the characteristics of a tensor such as:
    
* Its shape.
* The type of the data it contains.
* The device where it is held in memory (CPU/GPU).

In [ ]:
print("Shape:    {}".format(torch_tensor_ones.shape))
print("Datatype: {}".format(torch_tensor_ones.dtype))
print("Device:   {}".format(torch_tensor_ones.device))

The access to the data is similar to numpy:

* indices
* **range** of indices: begin:end:step.

In [ ]:
torch_tensor_ones[:,1] = -1
print(torch_tensor_ones)

Two very useful operators:
    
* **concatenation** of tensors;
* **reshaping** of a given tensor.

In [ ]:
print(torch_tensor_ones.shape,torch_tensor_rand.shape)
torch_tensor_concatenated = torch.cat([torch_tensor_ones, torch_tensor_rand], dim=1) # Try 1,2
print(torch_tensor_concatenated)

In [ ]:
print(torch_tensor_concatenated.shape)
torch_tensor_concatenated=torch_tensor_concatenated.reshape((2,10))
print(torch_tensor_concatenated)
print(torch_tensor_concatenated.shape)


In the implementations you will play with, you will perform mathematical operations on tensors. There are many of them!

In [ ]:
# Element-wise operations
torch_tensor = torch.rand_like(torch_tensor_concatenated, dtype=torch.float) 

print(torch_tensor_concatenated*torch_tensor) 
print(torch_tensor_concatenated+torch_tensor)
print(torch_tensor_concatenated-torch_tensor)
print(torch_tensor_concatenated/torch_tensor)

# Matrix multiplication
print(torch_tensor.matmul(torch_tensor.T))
print(torch_tensor @ torch_tensor.T)

Many functions apply some mathematical functions involving one or more dimensions in the tensors:
    
* mean
* max
* sum
* cumsum

For all of these, you need to specify the dimension(s) along which the operator applies.

In [ ]:
print(torch_tensor_concatenated)
print(torch.mean(torch_tensor_concatenated)) # No axis applies the operation to all the elements
print(torch.mean(torch_tensor_concatenated,axis=0))

**Exercise**:

* Generate a random $3\times 1000 \times 3$ tensor $\mathbf T$ with values distributed as a uniform distribution between -10 and 10. 
* Produce a $3 \times 3$ matrix $\mathbf M$ deduced from $\mathbf T$ by summing values along the second dimension.
* Compute $\mathbf M^3$. 

In [ ]:
t = torch.rand((3, 1000, 3)) * 20 - 10
s = torch.sum(t, axis = 1)
c = torch.pow(s, 3)
print(s @ (s @ s))
print(c)

# Auto grad

The main interest in using libraries such as Pytorch, Tensorflow, JAX is to have a transparent implementation of the **computation of the gradients** of the user-defined loss function. 

All of these propose a mechanism of **auto-differentiation** (auto-grad), to perform this task.

We recall that most supervised ML systems try to find a **parametrized funtion** $f_\theta$, parameterized by some parameters $\theta$,

$$
\mathbf y = f_\theta(\mathbf x),
$$

being given a number of examples (a.k.a. the training data) $\{\mathbf x_i,\mathbf y_i\}$.

To adjust our model, we need to define a loss function $\mathcal L()$ between the **expected output values** and our **model output values**:


$$
\theta^* = \arg\min_\theta \sum_i \mathcal L(\mathbf y_i,f_\theta(\mathbf x_i)).
$$


For example, in regression problems:

$$
\mathcal L(\mathbf y,f_\theta(\mathbf x))=\|\mathbf y-f_\theta(\mathbf x)\|^2.
$$


By using gradient descent:

$$
\theta' = \theta - \alpha\nabla_\theta \sum_i \mathcal L(\mathbf y_i,f_\theta(\mathbf x_i)).
$$

Auto-grad allows to compute this gradient.

In the DL jargon, this parameterized function is called a **model**.

We will see later how to **build** our models; first, you have to know that Pytorch has a number of **predefined models**.

In [ ]:
import torch,torchvision
from torchvision.models import resnet18

# A famous model which is predefined in Pytorch
model= resnet18()

# Dummy data: Note that there are 10 data: these is called a "batch"
X      = torch.rand(10, 3, 64, 64) # This is a group of 10 random color images
Y_true = torch.rand(10, 1000)      # We generate random target values

In [ ]:
# This is the forward pass:
Y_pred = model(X)
Y_pred.shape

In [ ]:
errors = (Y_pred - Y_true)**2 
print(errors.shape) # Note the size

# Sum all the squared differences
loss = errors.sum()

print(loss.shape, loss) # Note the size
loss.backward() # backpropagation is done here

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9) # model.parameters() is like theta - Stochastic gradient descent
#lr is learning rate for deep learning; it's called step for machine learning
optim.step() # apply one step of gradient descent

# Setting up a neural network

You define your model as a class inheriting from nn.Module.

It needs:

* a constructor (__init__ method)
* a forward method, that implements the forwaard pass of your network.

Typically the constructor defines **instances** of the layers you will use in your network (convolutional, fully connected...).

Examples:
* nn.Linear
* nn.Conv2d

Then the **forward** method details how we apply these layers to our data.

Often it follows a functional form:

```
x = f1(x)
x = f2(x)
x = f3(x)
return x
```




In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)


# A complete image classification example on CIFAR10

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

First step is to **prepare your data**.

There are a number of builtin functions:

* to load some very standard datasets (e.g., CIFAR10).
* to encapsulate them into a **"data loader"** that allows us to cycle over the data **batch after batch** (e.g. in training).

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

# Form the training dataset
trainset   = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader= torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Form the testing dataset
testset    = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Let us see what these images look like with matplotlib.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Functions to show an image
def imshow(img):
    img = img*0.5 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# Take the first batch and show the images
dataiter       = iter(trainloader)
images, labels = next(dataiter)
print(images.shape, labels.shape)
# Show images
matplotlib.rcParams['figure.figsize'] = [14, 7]
imshow(torchvision.utils.make_grid(images,nrow = 8))

# Print their corresponding classes
print(' '.join('{}'.format(classes[labels[j]]) for j in range(batch_size)))


Now we will define a network to **perform classification**, i.e. attribute to each possible class (among the 10) a probablity.

* Its **input** is a batch of 3x32x32 images.
* Its **output** is a batch of 10x1 vectors.
* We use convolutional layers to extract features from the image, at different scales.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class imageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # 3 input image channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # produce a batch of  6 x 14 x 14 features
        x = self.pool(F.relu(self.conv2(x)))  # produce a batch of 16 x 5 x  5 features
        x = torch.flatten(x, 1)               # flatten to produce a batch of vectors
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

classifier = imageClassifier()

In [ ]:
import torch.optim as optim

lossFunction = nn.CrossEntropyLoss()
optimizer    = optim.Adam(classifier.parameters(), lr=0.0015)

In [ ]:
losses = []
# Each pass on this cycle will see all the batches from the training dataset
for epoch in range(10):
    running_loss = 0.0
    for i, batch in enumerate(trainloader, 0): # enumerate(sequence, start = 0)
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass for the whole batch
        outputs = classifier(inputs)
        
        # Evaluate the loss 
        loss = lossFunction(outputs, labels)
        
        # Apply backpropagation: compute the gradient of the loss
        loss.backward()
        
        # Apply a gradient descent step: parameters = parameters - alpha * gradient
        optimizer.step()

        # Dislay loss values
        running_loss += loss.item()
        if i % 1000 ==  999:    # print every 1000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.3f}')
            losses.append([epoch,running_loss / 1000.0])
            running_loss = 0.0


In [ ]:
losses = np.array(losses)
plt.plot(losses[:,0],losses[:,1])

In [ ]:
PATH = './classifier.pth'
torch.save(classifier.state_dict(), PATH)

In [ ]:
batch          = iter(testloader)
images, labels = next(batch)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))
_, predicted = torch.max(classifier(images), 1)
print('Predicted:   ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(batch_size)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for batch in testloader:
        images, labels = batch
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(classifier(images).data, 1)
        total   += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total}%')


**Exercise**:

Recast the previous example as a ~~dog vs. non dog~~ animal vs non animal classification problem.    

Use 

https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html

# Image classifier model of two classes

In [ ]:
import torch
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Functions to show an image
def imshow(img):
    img = img*0.5 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
def transformLabels(lbl):
    animal = [2, 3, 4, 5, 6, 7]
    return torch.Tensor([lbl in animal])

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

# Form the training dataset
trainset   = torchvision.datasets.CIFAR10(root='../data/CIFAR10', train=True, download=True, transform=transform, target_transform = transformLabels)
trainloader= torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Form the testing dataset
testset    = torchvision.datasets.CIFAR10(root='../data/CIFAR10', train=False, download=True, transform=transform, target_transform = transformLabels)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

classes = ("animaln't", 'animal')

In [ ]:
# Take the first batch and show the images
dataiter       = iter(trainloader)
images, labels = next(dataiter)

In [ ]:
# Show images
matplotlib.rcParams['figure.figsize'] = [14, 7]
imshow(torchvision.utils.make_grid(images,nrow = 8))
# Print their corresponding classes
print(' '.join('{}'.format(classes[int(labels[j])]) for j in range(batch_size)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class animalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc4 = nn.Linear(10, 1)
        self.fc5 = nn.Softmax()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # produce a batch of  6 x 14 x 14 features
        x = self.pool(F.relu(self.conv2(x)))  # produce a batch of 16 x 5 x  5 features
        x = torch.flatten(x, 1)               # flatten to produce a batch of vectors
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

classifier = animalClassifier()

In [ ]:
import torch.optim as optim

lossFunction = nn.BCELoss()
optimizer    = optim.Adam(classifier.parameters(), lr=0.0015)

In [ ]:
losses = []
# Each pass on this cycle will see all the batches from the training dataset
for epoch in range(10):
    running_loss = 0.0
    for i, batch in enumerate(trainloader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass for the whole batch
        outputs = classifier(inputs)
        # Evaluate the loss 
        loss = lossFunction(outputs, labels)
        # Apply backpropagation: compute the gradient of the loss
        loss.backward()
        
        # Apply a gradient descent step: parameters = parameters - alpha * gradient
        optimizer.step()

        # Dislay loss values
        running_loss += loss.item()
        if i % 1000 ==  999:    # print every 1000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.3f}')
            losses.append([epoch,running_loss / 1000.0])
            running_loss = 0.0

In [ ]:
losses = np.array(losses)
plt.plot(losses[:,0],losses[:,1])

In [ ]:
batch          = iter(testloader)
images, labels = next(batch)
# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[int(labels[j])]:5s}' for j in range(batch_size)))
_, predicted = torch.max(classifier(images), 1)
print('Predicted:   ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(batch_size)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for batch in testloader:
        images, labels = batch
        # the class with the highest energy is what we choose as prediction
        predicted = torch.where(classifier(images).data >= 0.5, 1, 0)
        total   += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total}%')

# CPU/GPU

You can check what devices your system has:

In [ ]:
# Access to CUDA
device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

To use CUDA:

* Use the **to** method to move the data/model to the GPU
* Use the same method to get it back to CPU.